In [23]:
import pandas as pd
import numpy as np

In [24]:
data_set = pd.read_csv('../data/TrueFX/EUR-USD/datos_procesados_OHLC_5T_EURUSD-2018-01.csv', parse_dates=True,
                      infer_datetime_format=True,index_col=0)

In [25]:
data_set[:10]

,open,high,low,bid,ask
2018-01-02 00:00:00,1.20094,1.20094,1.20011,1.20015,1.20110
2018-01-02 00:05:00,1.20015,1.20062,1.20011,1.20047,1.20084
2018-01-02 00:10:00,1.20039,1.20098,1.20035,1.20082,1.20126
2018-01-02 00:15:00,1.20080,1.20097,1.20025,1.20035,1.20091
2018-01-02 00:20:00,1.20034,1.20057,1.20031,1.20049,1.20085
2018-01-02 00:25:00,1.20049,1.20054,1.20045,1.20047,1.20085
2018-01-02 00:30:00,1.20048,1.20093,1.20045,1.20079,1.20113
2018-01-02 00:35:00,1.20079,1.20163,1.20073,1.20135,1.20155
2018-01-02 00:40:00,1.20129,1.20145,1.20129,1.20143,1.20158
2018-01-02 00:45:00,1.20143,1.20150,1.20134,1.20149,1.20163


In [26]:
%run ../code/PNLEstimatorWrapper.py
%run ../code/EUtilities.py
EU = EUtilities()

In [27]:
window = 7
X, y_reg, bt = EU.build_dataset(data_set,window=window,bid_col='bid',binary_target=True,PNL=True)
X.shape

(6232, 9)

In [28]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,-0.00076,0.00022
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,-0.00023,-0.00012
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,-0.00020,-0.00009
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00017


In [29]:
X['open'] = data_set.filter(X.index,axis=0).open
X['high'] = data_set.filter(X.index,axis=0).high
X['low'] = data_set.filter(X.index,axis=0).low

In [30]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1,open,high,low
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,-0.00076,0.00022,1.20048,1.20093,1.20045
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,-0.00023,-0.00012,1.20079,1.20163,1.20073
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,-0.00020,-0.00009,1.20129,1.20145,1.20129
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030,1.20143,1.20150,1.20134
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00017,1.20150,1.20150,1.20131


In [33]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
wc0 = 0.06 #peso de la clase 0
wc1 = 0.06 #peso de la clase 1
wc2 = 0.44 #peso de la clase 2
wc3 = 0.44 #peso de la clase 3
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(priors=(wc0,wc1,wc2,wc3)), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 5T ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('longest_buy: ', np.max(result_GNB[:,3]))
print('longest_sell: ', np.max(result_GNB[:,4]))
print('%correct_1: ', np.mean(result_GNB[:,5]))
print('%correct_0: ', np.mean(result_GNB[:,6]))
print('%correct_3: ', np.mean(result_GNB[:,7]))
print('%correct_2: ', np.mean(result_GNB[:,8]))
print('% correct total: ', np.mean(result_GNB[:,5]+result_GNB[:,6]+result_GNB[:,7]+result_GNB[:,8]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2']), index=dates)
wc0 = np.ones(len(result_GNB))*wc0
wc1 = np.ones(len(result_GNB))*wc1
wc2 = np.ones(len(result_GNB))*wc2
wc3 = np.ones(len(result_GNB))*wc3

df_GNB.insert(loc=9,column='wc0',value=wc0)
df_GNB.insert(loc=10,column='wc1',value=wc1)
df_GNB.insert(loc=11,column='wc2',value=wc2)
df_GNB.insert(loc=12,column='wc3',value=wc3)

print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_5T_weights_GNB_NC4_OHLC_wsize7_2018-01.csv')

--------- GaussianNB 5T ----------------
average PNL :  -0.00134086956522
average buy:  102.869565217
average sell:  119.304347826
longest_buy:  286.0
longest_sell:  288.0
%correct_1:  0.111331394299
%correct_0:  0.0174429397168
%correct_3:  0.0584582903405
%correct_2:  0.0739824127962
% correct total:  0.261215037152
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-06  0.00000    1.0    0.0           0.0            1.0    0.000000   
2018-01-08  0.00000  245.0   34.0           0.0          279.0    0.000000   
2018-01-09 -0.01243  283.0    0.0         283.0            0.0    0.424028   
2018-01-10 -0.00266  261.0   20.0         148.0           73.0    0.284698   
2018-01-11  0.00000  177.0  105.0           0.0          282.0    0.000000   
2018-01-12  0.00000    0.0  280.0           0.0          280.0    0.000000   
2018-01-13 -0.00138    1.0    0.0           1.0            0.0    0.000000   
2018-01-15  0.0000

In [36]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree',weights='distance'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('--------- KNeighbors 5T ----------------')
print('average PNL : ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('longest_buy: ', np.max(result_KNC[:,3]))
print('longest_sell: ', np.max(result_KNC[:,4]))
print('%correct_1: ', np.mean(result_KNC[:,5]))
print('%correct_0: ', np.mean(result_KNC[:,6]))
print('%correct_3: ', np.mean(result_KNC[:,7]))
print('%correct_2: ', np.mean(result_KNC[:,8]))
print('% correct total: ', np.mean(result_KNC[:,5]+result_KNC[:,6]+result_KNC[:,7]+result_KNC[:,8]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2']), index=dates)
wc0 = 'distance'
wc1 = 'distance'
wc2 = 'distance'
wc3 = 'distance'

df_KNC.insert(loc=9,column='wc0',value=wc0)
df_KNC.insert(loc=10,column='wc1',value=wc1)
df_KNC.insert(loc=11,column='wc2',value=wc2)
df_KNC.insert(loc=12,column='wc3',value=wc3)

print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_5T_weights_KNC_NC4_OHLC_wsize7_2018-01.csv')

--------- KNeighbors 5T ----------------
average PNL :  -0.0105208695652
average buy:  112.739130435
average sell:  109.434782609
longest_buy:  170.0
longest_sell:  242.0
%correct_1:  0.172038244062
%correct_0:  0.168681880567
%correct_3:  0.00200816160684
%correct_2:  0.0020071678667
% correct total:  0.344735454102
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-06 -0.00057    1.0    0.0           1.0            0.0    0.000000   
2018-01-08 -0.01172  222.0   57.0         128.0            9.0    0.326165   
2018-01-09 -0.01036  262.0   21.0         170.0            4.0    0.402827   
2018-01-10 -0.01204  122.0  159.0          20.0           46.0    0.124555   
2018-01-11 -0.01317   80.0  202.0           8.0           23.0    0.117021   
2018-01-12 -0.02283   34.0  246.0           5.0          162.0    0.064286   
2018-01-13 -0.00138    1.0    0.0           1.0            0.0    0.000000   
2018-01-15 -0.00647

In [73]:
from sklearn.tree import DecisionTreeClassifier


wc0 = 0.2 #peso de la clase 0
wc1 = 0.2 #peso de la clase 1
wc2 = 0.3 #peso de la clase 2
wc3 = 0.3 #peso de la clase 3
est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(class_weight={0:wc0,1:wc1,2:wc2,3:wc3})
                              , PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('--------- DecisionTree 5T ----------------')
print('average PNL : ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('longest_buy: ', np.max(result_DTC[:,3]))
print('longest_sell: ', np.max(result_DTC[:,4]))
print('%correct_1: ', np.mean(result_DTC[:,5]))
print('%correct_0: ', np.mean(result_DTC[:,6]))
print('%correct_3: ', np.mean(result_DTC[:,7]))
print('%correct_2: ', np.mean(result_DTC[:,8]))
print('% correct total: ', np.mean(result_DTC[:,5]+result_DTC[:,6]+result_DTC[:,7]+result_DTC[:,8]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2']), index=dates)
wc0 = np.ones(len(result_DTC))*wc0
wc1 = np.ones(len(result_DTC))*wc1
wc2 = np.ones(len(result_DTC))*wc2
wc3 = np.ones(len(result_DTC))*wc3

df_DTC.insert(loc=9,column='wc0',value=wc0)
df_DTC.insert(loc=10,column='wc1',value=wc1)
df_DTC.insert(loc=11,column='wc2',value=wc2)
df_DTC.insert(loc=12,column='wc3',value=wc3)

print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_5T_weights_DTC_NC4_OHLC_wsize7_2018-01.csv')

--------- DecisionTree 5T ----------------
average PNL :  -0.00728956521739
average buy:  110.173913043
average sell:  112.0
longest_buy:  206.0
longest_sell:  244.0
%correct_1:  0.188602648103
%correct_0:  0.148612743075
%correct_3:  0.00463285049881
%correct_2:  0.00417451927127
% correct total:  0.346022760949
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-06 -0.00057    1.0    0.0           1.0            0.0    0.000000   
2018-01-08 -0.00225  226.0   53.0          14.0          165.0    0.053763   
2018-01-09 -0.01105  106.0  177.0          42.0          164.0    0.123675   
2018-01-10 -0.00577  153.0  128.0          18.0           16.0    0.188612   
2018-01-11 -0.00753  114.0  168.0           8.0           15.0    0.148936   
2018-01-12 -0.00498   23.0  257.0           3.0          177.0    0.032143   
2018-01-13  0.00000    0.0    1.0           0.0            1.0    0.000000   
2018-01-15 -0.00304  25

In [81]:
from sklearn.ensemble import RandomForestClassifier

wc0 = 1 #peso de la clase 0
wc1 = 1 #peso de la clase 1
wc2 = 2 #peso de la clase 2
wc3 = 2 #peso de la clase 3

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3,
                                                     class_weight={0:wc0,1:wc1,2:wc2,3:wc3})
                              ,PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('--------- RandomForest 5T ----------------')
print('average PNL : ', np.mean(result_RFC[:,0]))
print('average buy: ', np.mean(result_RFC[:,1]))
print('average sell: ', np.mean(result_RFC[:,2]))
print('longest_buy: ', np.max(result_RFC[:,3]))
print('longest_sell: ', np.max(result_RFC[:,4]))
print('%correct_1: ', np.mean(result_RFC[:,5]))
print('%correct_0: ', np.mean(result_RFC[:,6]))
print('%correct_3: ', np.mean(result_RFC[:,7]))
print('%correct_2: ', np.mean(result_RFC[:,8]))
print('% correct total: ', np.mean(result_RFC[:,5]+result_RFC[:,6]+result_RFC[:,7]+result_RFC[:,8]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2']), index=dates)
wc0 = np.ones(len(result_RFC))*wc0
wc1 = np.ones(len(result_RFC))*wc1
wc2 = np.ones(len(result_RFC))*wc2
wc3 = np.ones(len(result_RFC))*wc3

df_RFC.insert(loc=9,column='wc0',value=wc0)
df_RFC.insert(loc=10,column='wc1',value=wc1)
df_RFC.insert(loc=11,column='wc2',value=wc2)
df_RFC.insert(loc=12,column='wc3',value=wc3)

print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_5T_weights_RFC_NC4_OHLC_wsize7_2018-01.csv')

--------- RandomForest 5T ----------------
average PNL :  -0.00737173913044
average buy:  99.9130434783
average sell:  122.260869565
longest_buy:  168.0
longest_sell:  244.0
%correct_1:  0.15083053752
%correct_0:  0.180468057792
%correct_3:  0.00170812181706
%correct_2:  0.00356812862979
% correct total:  0.336574845759
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-06 -0.00057    1.0    0.0           1.0            0.0    0.000000   
2018-01-08 -0.00872  225.0   54.0           9.0          124.0    0.075269   
2018-01-09 -0.01061  257.0   26.0         168.0            4.0    0.395760   
2018-01-10 -0.00673   85.0  196.0           7.0           36.0    0.113879   
2018-01-11 -0.01140   95.0  187.0           6.0           48.0    0.148936   
2018-01-12 -0.02445   21.0  259.0           4.0          176.0    0.035714   
2018-01-13 -0.00138    1.0    0.0           1.0            0.0    0.000000   
2018-01-15 -0.00